# Simple Chain 

In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os

In [26]:
load_dotenv()
API_KEY=os.getenv("API_KEY")
api=os.getenv('mapi')

In [5]:
prompt=PromptTemplate(
    template="Generate 5 intresting fact about topic {topic}",
    input_variables=['topic']

)

In [11]:
llm = ChatGoogleGenerativeAI(
    model ="gemini-2.5-flash",
    google_api_key = API_KEY
)
parser=StrOutputParser() # parser takes output of llm and turns to sturctured format
# StrOutputParser() returns model output as 'string'
chain = prompt | llm | parser

In [13]:
result=chain.invoke({"topic":"AI"})

In [14]:
print(result)

Here are 5 interesting facts about Artificial Intelligence:

1.  **AI's Name Was Coined in 1956:** The term "Artificial Intelligence" was first coined by computer scientist John McCarthy at the Dartmouth Conference in 1956. This landmark event is often considered the birth of AI as a field of study, bringing together researchers to discuss the possibility of machines simulating human intelligence.

2.  **You Interact with AI Daily (Often Without Realizing It):** From the personalized recommendations on Netflix and Spotify, to the spam filters in your email, the voice assistants on your phone (Siri, Alexa, Google Assistant), and even the navigation apps optimizing your route – AI is constantly working behind the scenes to make our digital lives more efficient and tailored.

3.  **Current AI is "Narrow AI":** Despite sci-fi portrayals, today's AI is primarily "narrow AI" or "weak AI." This means it's designed to perform specific tasks extremely well (like playing chess, recognizing faces

In [ ]:
chain.get_graph().print_ascii()

      +-------------+      
      | PromptInput |      
      +-------------+      
             *             
             *             
             *             
    +----------------+     
    | PromptTemplate |     
    +----------------+     
             *             
             *             
             *             
+------------------------+ 
| ChatGoogleGenerativeAI | 
+------------------------+ 
             *             
             *             
             *             
    +-----------------+    
    | StrOutputParser |    
    +-----------------+    
             *             
             *             
             *             
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  



### Flow of `prompt | llm | parser`

1. **`prompt`**  
   - Provides the input text or question.  
   - Example: `"Summarize this article in simple language."`  

2. **`llm = ChatGoogleGenerativeAI(...)`**  
   - Language model object (Gemini 2.5).  
   - Takes the prompt and **generates a response**.  

3. **`parser = StrOutputParser()`**  
   - Takes the output from the model and **returns it as a plain string**.  

4. **`chain = prompt | llm | parser`**  
   - Connects all three steps.  
   - Input → LLM → Parser → Final output.  

-----------------------------------------------------------




# Sequential chain

In [20]:
seqprompt1=PromptTemplate(
    template="Generate a detailed report on {topic}",
    input_variables=['topic']
)

In [21]:
seqprompt2=PromptTemplate(
    template="Generate 5 point summary from the following text {text} ",
    input_variables=['text']
)

In [22]:
chain=seqprompt1 | llm | parser | seqprompt2 | llm |parser

In [23]:
resultseq=chain.invoke({"topic":"Art"})

In [24]:
print(resultseq)

Here is a 5-point summary of the detailed report on Art:

1.  **Defining Art:** The report emphasizes that "Art" is a complex and elusive concept with no single universal definition, largely due to its subjective nature, though it commonly involves creativity, expression, communication, and aesthetic appeal.
2.  **Historical Evolution:** It traces art's extensive historical trajectory from prehistoric expressions through ancient, medieval, renaissance, and modern movements, highlighting a continuous narrative of innovation, adaptation, and reflection of human civilization.
3.  **Diverse Forms:** Art manifests in myriad forms and disciplines, broadly categorized into visual arts (e.g., painting, sculpture, digital art), performing arts (e.g., music, dance, theatre), literary arts, applied arts & design, and new media, showcasing its vast manifestations.
4.  **Multifaceted Functions:** Art serves a wide range of purposes within societies, including providing aesthetic pleasure, facilitat

In [25]:
chain.get_graph().print_ascii()

      +-------------+      
      | PromptInput |      
      +-------------+      
             *             
             *             
             *             
    +----------------+     
    | PromptTemplate |     
    +----------------+     
             *             
             *             
             *             
+------------------------+ 
| ChatGoogleGenerativeAI | 
+------------------------+ 
             *             
             *             
             *             
    +-----------------+    
    | StrOutputParser |    
    +-----------------+    
             *             
             *             
             *             
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  
             *             
             *             
             *             
    +----------------+     
    | PromptTemplate |     
    +----------------+     
             *             
             *             
             *      

# Parallel Chain

In [27]:
from langchain_mistralai.chat_models import ChatMistralAI

llm1 = ChatGoogleGenerativeAI(
    model ="gemini-2.5-flash",
    google_api_key = API_KEY
)
llm2=ChatMistralAI(
    model="mistral-small-latest",  
    mistral_api_key=api,
)


In [28]:
prompt1=PromptTemplate(
    template="Generate notes from follwing topic {topic}",
    input_variables=['topic']
)
prompt2=PromptTemplate(
    template="Generate 5 quiz from the topic {topic}",
    input_variables=['topic']
)
prompt3=PromptTemplate(
    template="Merge the provided quiz and note into single documents {notes}\n {quiz}",
    input_variables=['notes','quiz']
)

In [29]:
parser=StrOutputParser()

In [ ]:
from langchain.schema.runnable import RunnableParallel
# with this we can parallely multiple chain execute 

In [31]:
parallel_chain= RunnableParallel({
    'notes': prompt1 | llm1 | parser,
    "quiz": prompt2 | llm2 | parser
})

In [ ]:
merge_chain = prompt3 | llm1 | parser 

In [ ]:
chain = parallel_chain | merge_chain 

In [35]:
result=chain.invoke({"topic":"unemployment"})

In [36]:
print (result)

Here is the merged document, combining the comprehensive notes on Unemployment with the quiz questions for assessment.

---

# Unemployment: Key Concepts, Analysis, and Assessment

This document provides comprehensive notes on the topic of Unemployment, structured for clarity and easy understanding, followed by a set of quiz questions to test your knowledge.

---

## Unemployment: Key Concepts & Analysis

### I. Definition & Measurement

*   **Definition:** A situation where individuals who are **able to work**, **willing to work**, and **actively seeking employment** cannot find a job.
*   **Labor Force:** Consists of all employed and unemployed people.
    *   **Employed:** People who work for pay (full-time or part-time) or work in their own business.
    *   **Unemployed:** People who are not working but are available for work and have actively looked for work in the past four weeks.
*   **Not in Labor Force:** Individuals not employed and not actively seeking work (e.g., retirees,

In [38]:
# or can be done done to passed text like this 
text=""" Google APIs are Application Programming Interfaces developed by Google that enable communication with various Google services and facilitate their integration into other applications and services. These APIs allow developers to programmatically access and interact with data and functionalities offered by Google products.
Key aspects of Google APIs:
Integration with Google Services:
They provide programmatic access to a wide range of Google services, including Search, Gmail, Google Maps, Google Drive, Google Sheets, Google Docs, YouTube, and many Google Cloud Platform services.
Enhanced Functionality:
Developers can leverage Google APIs to add features and functionalities to their own applications, such as embedding maps, managing documents, sending emails, or integrating with cloud computing resources.
Developer Tools and Resources:
Google provides various tools and resources to assist developers in using these APIs, including client libraries for different programming languages (e.g., Python, Java, JavaScript), comprehensive documentation, and the APIs Explorer for testing API methods.
Authentication and Authorization:
Access to most Google APIs requires proper authentication and authorization to ensure secure access to user data and resources. This often involves using API keys, OAuth 2.0, or service accounts.
Google Cloud APIs:
A significant subset of Google APIs falls under Google Cloud Platform, providing interfaces to services like computing, networking, storage, and machine learning, enabling the development of scalable and powerful cloud-based applications.
"""
result=chain.invoke({"topic":text})


In [39]:
print(result)

Here is the merged document, combining the notes and the quiz on Google APIs:

---

## Google APIs: Application Programming Interfaces

### Notes

**Definition:**
*   Developed by Google.
*   Enable communication & integration with various Google services.
*   Allow developers programmatic access to data & functionalities of Google products.

**Key Aspects:**

1.  **Integration with Google Services:**
    *   Provide programmatic access to a wide range of services.
    *   **Examples:** Search, Gmail, Google Maps, Google Drive, Google Sheets, Google Docs, YouTube, Google Cloud Platform (GCP) services.

2.  **Enhanced Functionality:**
    *   Developers can add features and functionalities to their own applications.
    *   **Examples:** Embedding maps, managing documents, sending emails, integrating with cloud resources.

3.  **Developer Tools & Resources:**
    *   Google provides assistance for developers.
    *   **Includes:**
        *   Client libraries (e.g., Python, Java, JavaSc

In [40]:
chain.get_graph().print_ascii()

                  +---------------------------+                
                  | Parallel<notes,quiz>Input |                
                  +---------------------------+                
                     ****               ****                   
                  ***                       ***                
                **                             **              
    +----------------+                    +----------------+   
    | PromptTemplate |                    | PromptTemplate |   
    +----------------+                    +----------------+   
             *                                      *          
             *                                      *          
             *                                      *          
+------------------------+                 +---------------+   
| ChatGoogleGenerativeAI |                 | ChatMistralAI |   
+------------------------+                 +---------------+   
             *                          

# Conditional Chain

In [41]:
llm1 = ChatGoogleGenerativeAI(
    model ="gemini-2.5-flash",
    google_api_key = API_KEY
)


# 🧾 Why Use `PydanticOutputParser` in LangChain?

## 1. The Problem: Unstructured LLM Output
LLM outputs are not always consistent when asked for sentiment.  

Examples:
```text
"negative"
"The sentiment is negative."
"NEGATIVE"
"It's definitely negative sentiment."



This is fine if a human is reading, but bad if another chain or program depends on consistent output.

In [60]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import Literal
# these are done becoz we dont know exact ouput of llm can give The sentiment is **negative**. , or only negative or Negative ,or positive similar way so we define this to make consistance ouput of llm since our next setp depend on output

class FeedBack (BaseModel):
    sentiment : Literal['positive','negative']=Field(description ="Give the sentiment of the feedback")

parser2=PydanticOutputParser(pydantic_object=FeedBack)
prompt1 = PromptTemplate(
    template="""Classify the sentiment of following feedback text as negative or positive. 
    Feedback: {feedback}
    {format_instruction}""",
    input_variables=["feedback"],
    partial_variables={"format_instruction": parser2.get_format_instructions()}
)



In [61]:
classifier_chain= prompt1 | llm | parser2


In [62]:
result=classifier_chain.invoke({"feedback":"i hate bugs"})
print(result)

sentiment='negative'


In [63]:
from langchain.schema.runnable import RunnableBranch
#Runnable branch allows us to use if else conditions in chains

In [64]:
from langchain.schema.runnable import RunnableLambda
# lambda funtion is converted to runnable so we can use as chain
"""branch = RunnableBranch([
    (condition1, chain1),
    (condition2, chain2),"""
prompt2 = PromptTemplate(
    template="Write appropriate response to positive feed back {feedback}",
    input_variables=['feedback']
)
prompt3 = PromptTemplate(
    template="Write appropriate response to negative feed back {feedback}",
    input_variables=['feedback']
)
branch_chain=RunnableBranch(
    (lambda x:x.sentiment=='positive',prompt2 | llm |parser),
    (lambda x:x .sentiment=='negative',prompt3 | llm |parser),
    RunnableLambda(lambda x: "Could not find sentiment")
)

In [65]:
chain=classifier_chain | branch_chain

In [67]:
print(chain.invoke({"feedback":"life is not so beautiful"}))

When responding to negative feedback, the goal is to acknowledge the customer's feelings, validate their experience, apologize sincerely, offer a solution or next step, and show that you value their input and are committed to improvement.

Here's a guide to crafting an appropriate response, along with examples for different scenarios:

---

**Key Principles for Responding to Negative Feedback:**

1.  **Acknowledge and Thank:** Start by thanking them for their feedback, even if it's negative. This shows you value their input.
2.  **Empathize and Apologize:** Show genuine empathy for their negative experience and sincerely apologize for any inconvenience, frustration, or disappointment caused. Avoid making excuses.
3.  **Validate Their Experience:** Let them know you understand *why* they're upset.
4.  **Take Responsibility:** Even if it's not directly your fault, take responsibility for the overall negative experience.
5.  **Address the Specific Issue (if possible):** If they mentioned 

In [68]:
chain.get_graph().print_ascii()

      +-------------+      
      | PromptInput |      
      +-------------+      
             *             
             *             
             *             
    +----------------+     
    | PromptTemplate |     
    +----------------+     
             *             
             *             
             *             
+------------------------+ 
| ChatGoogleGenerativeAI | 
+------------------------+ 
             *             
             *             
             *             
 +----------------------+  
 | PydanticOutputParser |  
 +----------------------+  
             *             
             *             
             *             
        +--------+         
        | Branch |         
        +--------+         
             *             
             *             
             *             
     +--------------+      
     | BranchOutput |      
     +--------------+      
